#### DESAFIO Nº2

Alumna: Lara Rosenberg

#### CONSIGNA
Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
Probar términos de interés y explicar similitudes en el espacio de embeddings.
Intentar plantear y probar tests de analogías.
Graficar los embeddings resultantes.
Sacar conclusiones.

In [1]:
pip install python-docx

In [2]:
pip install unidecode

In [1]:
# Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from unidecode import unidecode
from gensim.models.callbacks import CallbackAny2Vec
from docx import Document
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import plotly.express as px

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Para este trabajo, decidí trabajar con un resumen del libro Harry Potter y la piedra filosofal, de J.K. Rowling.

In [2]:
# Cargamos el archivo de un word
documento = Document('resumen_harry_potter_1.docx')

# Leemos el contenido del documento
texto = []
for parrafo in documento.paragraphs:
    texto.append(parrafo.text)

# Unimos los parrafos en un solo string
texto_completo = '\n'.join(texto)

print(texto_completo)

La novela comienza con la descripción de los Dursley, una familia de clase media que vive en el numero cuatro de Privet Drive, en Little Whinging, Surrey. Vernon y Petunia Dursley están constantemente preocupados de que alguien pueda enterarse de que la hermana de Petunia, Lily, es una bruja, y estos temores son puestos a prueba cuando el hijo de ella, Harry, es dejado en la puerta de su casa junto a una carta. Lily y su esposo, James Potter, han sido asesinados por Voldemort, un Mago Oscuro, pero cuando este ha intentado matar a Harry, su poder por alguna razón se quebró. Harry se transforma entonces en la única persona que ha sobrevivido a la maldición asesina, y el único signo que ha quedado de su encuentro con Voldemort es su cicatriz en forma de rayo en la frente. Como Harry no tiene ningún otro pariente, Albus Dumbledore, el director del colegio Hogwarts de Magia y Hechicería, decide dejarlo con los Dursley hasta que Harry sea lo suficientemente mayor para concurrir a Hogwarts. T

In [3]:
# Como preprocesamiento del texto, eliminamos stopwords (palabras muy frecuentes como conectores) y eliminamos las tildes
stop_words = set(stopwords.words('spanish'))

secuencia_palabras = [
    [unidecode(palabra) for palabra in text_to_word_sequence(parrafo) if palabra not in stop_words]
    for parrafo in texto]

print(secuencia_palabras)

[['novela', 'comienza', 'descripcion', 'dursley', 'familia', 'clase', 'media', 'vive', 'numero', 'cuatro', 'privet', 'drive', 'little', 'whinging', 'surrey', 'vernon', 'petunia', 'dursley', 'constantemente', 'preocupados', 'alguien', 'pueda', 'enterarse', 'hermana', 'petunia', 'lily', 'bruja', 'temores', 'puestos', 'prueba', 'hijo', 'harry', 'dejado', 'puerta', 'casa', 'junto', 'carta', 'lily', 'esposo', 'james', 'potter', 'sido', 'asesinados', 'voldemort', 'mago', 'oscuro', 'intentado', 'matar', 'harry', 'poder', 'alguna', 'razon', 'quebro', 'harry', 'transforma', 'entonces', 'unica', 'persona', 'sobrevivido', 'maldicion', 'asesina', 'unico', 'signo', 'quedado', 'encuentro', 'voldemort', 'cicatriz', 'forma', 'rayo', 'frente', 'harry', 'ningun', 'pariente', 'albus', 'dumbledore', 'director', 'colegio', 'hogwarts', 'magia', 'hechiceria', 'decide', 'dejarlo', 'dursley', 'harry', 'suficientemente', 'mayor', 'concurrir', 'hogwarts', 'petunia', 'vernon', 'dudley', 'muggle', 'personas', 'mag

Generamos los tokens

In [4]:
# Crearmos el modelo generador de vectores, en este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=2,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [5]:
# Obtenemos el vocabulario con los tokens
w2v_model.build_vocab(secuencia_palabras)

In [6]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de documentos en el corpus:", w2v_model.corpus_count)

Cantidad de documentos en el corpus: 35


In [7]:
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de palabras distintas en el corpus: 202


In [8]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época, creamos esta funcion para incluirlo
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

Entrenamos los embeddings

In [9]:
# Entrenamos el modelo generador de vectores
w2v_model.train(secuencia_palabras,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 25293.953125
Loss after epoch 1: 25818.24609375
Loss after epoch 2: 14714.95703125
Loss after epoch 3: 8675.296875
Loss after epoch 4: 7267.1171875
Loss after epoch 5: 7096.6640625
Loss after epoch 6: 7204.1015625
Loss after epoch 7: 6822.5234375
Loss after epoch 8: 7411.203125
Loss after epoch 9: 7083.765625
Loss after epoch 10: 7335.5390625
Loss after epoch 11: 7140.4765625
Loss after epoch 12: 6967.921875
Loss after epoch 13: 7316.453125
Loss after epoch 14: 7229.25
Loss after epoch 15: 7199.875
Loss after epoch 16: 6888.34375
Loss after epoch 17: 7108.234375
Loss after epoch 18: 7317.484375
Loss after epoch 19: 7124.875
Loss after epoch 20: 7400.21875
Loss after epoch 21: 7493.546875
Loss after epoch 22: 6993.765625
Loss after epoch 23: 6871.34375
Loss after epoch 24: 7148.703125
Loss after epoch 25: 7527.171875
Loss after epoch 26: 7179.078125
Loss after epoch 27: 7167.59375
Loss after epoch 28: 7349.390625
Loss after epoch 29: 7185.953125
Loss after epoch 30: 

(23814, 65650)

Analizamos las palabras que más y menos se relacionan con palabras de interes y el por qué

In [10]:
# Palabras que mas se relacionan con Dumbledore:
w2v_model.wv.most_similar(positive=["dumbledore"], topn=5)

[('harry', 0.9990016222000122),
 ('explica', 0.9988055229187012),
 ('director', 0.9987824559211731),
 ('ultima', 0.9987255334854126),
 ('aunque', 0.9986613392829895)]

Podemos notar que en las palabras más relacionadas con Dumbledore está "director" (que es su puesto en Hogwarts), "Harry" y "explica", ya que es un personaje muy cercano a Harry Potter y que le suele enseñar varias cosas.

In [11]:
# Palabras que menos se relacionan con Harry:
w2v_model.wv.most_similar(negative=["harry"], topn=5)

[('tres', -0.9882661700248718),
 ('perro', -0.989778459072113),
 ('cabezas', -0.9904507398605347),
 ('centauro', -0.9910827875137329),
 ('corredor', -0.9913155436515808)]

Las palabras que salen tiene sentido que no esten relacionadas, ya que hacen alusion al perro de 3 cabezas y al centauro, que son otros personajes.

In [12]:
# Palabras que más se relacionan con piedra:
w2v_model.wv.most_similar(positive=["piedra"], topn=5)

[('filosofal', 0.9989685416221619),
 ('obtener', 0.9984893202781677),
 ('bolsillo', 0.9984751343727112),
 ('cuenta', 0.997785210609436),
 ('trata', 0.9975601434707642)]

Para el caso de piedra, vemos que las palabras que más se relacionan son "filosofal", "obtener" y "bolsillo" que es desde donde Harry saca la piedra filosofal.

In [13]:
# Palabras que más se relacionan con mago:
w2v_model.wv.most_similar(positive=["mago"], topn=5)

[('sido', 0.9986450672149658),
 ('oscuro', 0.9985371232032776),
 ('potter', 0.9983808398246765),
 ('poder', 0.998158872127533),
 ('pesar', 0.9977843761444092)]

En el caso de mago, es interesante que las palabras que más se vinculan son "Potter" (ya que Harry es un mago), "oscuro" (ya que Voldemort es un mago tenebroso) y "poder" (que es basicamente lo que busca Voldemort durante toda la saga).

In [14]:
# Palabras que mas se relacionan con voldemort:
w2v_model.wv.most_similar(positive=["voldemort"], topn=5)

[('frente', 0.998530387878418),
 ('entonces', 0.9983583092689514),
 ('pesar', 0.9982214570045471),
 ('poder', 0.9981600046157837),
 ('profesor', 0.9980496764183044)]

En el caso de Voldemort, las palabras que aparecen como mas relacionadas son "profesor" ya que Voldemort vivia a traves del profesor Quirrell, "frente" que es donde le deja la cicatriz a Harry Potter al intentar matarlo y "poder" ya que era un mago poderoso que buscaba tener todo el poder.
Despues se observan algunas palabras que no hacen referencia a él en particular ("entonces", "pesar")


Test de analogias

La analogia que vimos en clase es la de rey - hombre + mujer. Vamos a intentar de buscar si encontramos en el espacio de los embeddings alguna analogía

In [15]:
resultado = w2v_model.wv.most_similar(positive=['harry', 'voldemort'], negative=['gryffindor'], topn=1)
print(f"Analogía: harry - gryffindor + voldemort = {resultado[0][0]} (similitud: {resultado[0][1]})")

Analogía: harry - gryffindor + voldemort = profesor (similitud: 0.996084988117218)


Aca esperaba encontrar la palabra slytherin, pero no sucedió

In [16]:
resultado = w2v_model.wv.most_similar(positive=['harry', 'dudley'], negative=['mago'], topn=1)
print(f"Analogía: harry - mago + dudley = {resultado[0][0]} (similitud: {resultado[0][1]})")

Analogía: harry - mago + dudley = primo (similitud: 0.9963192939758301)


Aca esperaba encontrar que Harry era a mago lo que Dudley era a muggle, pero tampoco se verificó la analogía

Visualizamos agrupación de embeddings

In [17]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [18]:
# Graficamos los embedddings en 2D reduciendo la dimensionalidad con TSNE
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab")

Oservando a los embeddings en 2 dimensiones, podemos notar que se generan grupos de palabras que se encuentran altamente relacionadas.
A modo de ejemplo:
- Si buscamos "piedra" nos vamos a encontrar con "obtener", "filosofal", "bolsillo".
- Si miramos "gryffindor", nos aparece "slytherin", "casas" y "puntos" lo que tiene sentido ya que los estudiantes dentro del castillo se dividian en las distintas casas (entre ellas gryffindor y slytherin) y tenian que ganar puntos para sus casas.
- Si revisamos por ejemplo la palabra "quiddich", nos encontramos con que las palabras más cercanas en 2D son "partido", "busca", "snitch", lo que tiene total sentido ya que en los partidos de quiddich lo que se busca es atrapar la snitch dorada que hace que finalice la partida.
- Si buscamos "howarts" observamos que se encuentra muy cercana a "magia", "hechiceria", "colegio".
- Si revisamos la palabra "voldemort" vemos que sus palabras más cercanas son "poder", "oscuro", "profesor", "quirrell"

In [19]:
# Graficamos los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab")